In [34]:
import importlib

import torch
from constants import *
import utils
importlib.reload(utils)

CLASSIFIED_STATES = ['boredom', 'flow', 'frustration', 'neutral']

print("CUDA Available: ", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Current Device: ", device)

CUDA Available:  False
Current Device:  cpu


In [40]:
X_train_init, y_train_init = utils.load_testing_data(CLASSIFIED_STATES)
X_train_init.shape, y_train_init.shape

(torch.Size([5832, 25, 40]), torch.Size([5832, 4]))

In [41]:
X_train = X_train_init.mean(axis=1) # [:100, :]
y_train = y_train_init # [:100, :]
X_train.shape, y_train.shape

(torch.Size([5832, 40]), torch.Size([5832, 4]))

In [38]:
from sklearn.svm import SVC
from skl2onnx import to_onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx.helpers.onnx_helper import select_model_inputs_outputs

svm = None
svm_onnx = None
for i, y_class in enumerate(CLASSIFIED_STATES):
    print(f"Training for {y_class}")
    svm = SVC(kernel='linear', C=1.0, random_state=42, probability=True) 
    y = y_train[:, i]
    svm.fit(X_train, y)

    # export to onnx
    initial_type = [('Z', FloatTensorType([None, X_train.shape[1]]))]
    svm_onnx = to_onnx(svm, X_train, initial_types=initial_type)
    svm_onnx = select_model_inputs_outputs(svm_onnx, outputs=['probabilities'])

    with open(f"{EXPORTED_MODELS_PATH}/svm_{y_class}.onnx", "wb") as f:
        f.write(svm_onnx.SerializeToString())
        

Training for boredom
Training for flow
Training for frustration
Training for neutral


### Validate the results and compatre onnx with sklearn

In [39]:
# % pip install onnxruntime==1.18.1
import onnxruntime as rt
import numpy as np

example_x = X_train[y_train[:, 0] == 1][:3, :] # boredom has index 0
print(example_x.shape)

session = rt.InferenceSession(f"{EXPORTED_MODELS_PATH}/svm_neutral.onnx", providers=["CPUExecutionProvider"])
input_name = session.get_inputs()[0].name
label_name = [o.name for o in session.get_outputs()]

print(f'input={input_name}, output={label_name}')

print(f'onnx pred: {session.run(label_name, {input_name: example_x.numpy().astype(np.float32)})}')
print(f'sklearn pred: {svm.predict_proba(example_x)}')

input=Z, output=['probabilities']
onnx pred: [array([[0.8026046 , 0.19739535],
       [0.86141944, 0.13858062],
       [0.8250284 , 0.17497148]], dtype=float32)]
sklearn pred: [[0.80259995 0.19740005]
 [0.86142192 0.13857808]
 [0.82503909 0.17496091]]
